In [13]:
import tensorflow as tf
import pandas as pd
import numpy as np
import sklearn 
from keras.layers import Dense
from keras.models import Sequential
import os

In [14]:
!pip install pyyaml h5py

In [15]:
!pip install matplotlib

In [16]:
df1 = pd.read_csv('/home/saichaitanya/Chaitanya/CSV files/May3rd1percent_RLV_data 100 points2 .csv',header = None,names = ['h','v','s','omega','gamma','m','theta','Thrust','beta','time'])
df2 = pd.read_csv('/home/saichaitanya/Chaitanya/CSV files/May3rd1percent_RLV_data 100 points2_after428 .csv',header = None,names = ['h','v','s','omega','gamma','m','theta','Thrust','beta','time'])
df3 = pd.read_csv('/home/saichaitanya/Chaitanya/CSV files/May3rd1percent_RLV_data 100 points2_after450 .csv',header = None,names = ['h','v','s','omega','gamma','m','theta','Thrust','beta','time'])
df = pd.DataFrame()
df = pd.concat([df1,df2,df3],ignore_index=True)
df3=pd.DataFrame(df.iloc[:,0:3])
input = df3.values
output = df.values 
X = input[:,0:]
y = output[:,7:9]
# Individual Data
h = X[:,0]
v = X[:,1]
s = X[:,2]
# t = X[:,3]
T = y[:,0]
b = y[:,1]

In [17]:
from sklearn import preprocessing
X_norm = preprocessing.minmax_scale(X)
y_norm = preprocessing.minmax_scale(y)
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X_norm, y_norm, test_size=0.2, random_state=42)

In [18]:
def create_model(n):
    model = Sequential()
    model.add(Dense(n,input_shape=(3,),kernel_initializer='uniform'))
    model.add(Dense(n,kernel_initializer='uniform',activation = 'relu'))
    model.add(Dense(n,kernel_initializer='uniform',activation = 'tanh')) # since tanh has more nonlinearity we add it here, it also gives -ve values so , some layers which are not necessary will lead to 0 in next layer
    model.add(Dense(n,kernel_initializer='uniform',activation = 'relu'))
    model.add(Dense(2,kernel_initializer='uniform',activation = 'sigmoid'))
    return model

In [35]:
lr = 0.001;
for i in range(20):
    lrn = lr * (1 / (1 + 0.15 * i))
    print(lrn)
    lr=lrn;

0.001
0.0008695652173913045
0.0006688963210702342
0.0004613078076346443
0.0002883173797716527
0.0001647527884409444
8.671199391628652e-05
4.229853361770074e-05
1.9226606189863973e-05
8.181534548878288e-06
3.272613819551315e-06
1.2349486111514398e-06
4.410530754112285e-07
1.4950951708855202e-07
4.822887648017807e-08
1.4839654301593252e-08
4.364604206350956e-09
1.2294659736199878e-09
3.322881009783751e-10
8.630859765672078e-11


In [97]:
def lr_sch(epoch,lr):
    if (epoch>0) & (epoch<200):
      if epoch%50==0: # for every 100 epochs the learning rate varies as metioned. 
        return lr*np.exp(-0.45)
      else:
        return lr
    elif(epoch>200):
        if epoch%50==0:
            return lr*np.exp(-0.225);
        else:
            return lr
    else:
        return lr

In [110]:
for i in range(4,11,1):
  model = create_model(2**i)
  opt = tf.keras.optimizers.Adam(learning_rate = 0.001)
  loss = tf.keras.losses.mean_squared_error
  def lr_sch(epoch,lr):
    if (epoch>0) & (epoch<200):
      if epoch%50==0: # for every 100 epochs the learning rate varies as metioned. 
        return lr*np.exp(-0.45)
      else:
        return lr
    elif(epoch>200):
        if epoch%50==0:
            return lr*np.exp(-0.225);
        else:
            return lr
    else:
        return lr
  print("The Number of Hidden units used is: ",2**i)
  lr_scheduler = tf.keras.callbacks.LearningRateScheduler(lr_sch,verbose = 1)
    #     batchsize = 128
    # STEPS_PER_EPOCH = X_train.shape[0] / batchsize
    # save_period = 5
    #     cp_callback = tf.keras.callbacks.ModelCheckpoint(filepath=checkpoint_path,
    #                                               save_weights_only=True,
    #                                               verbose=1,save_freq=int(save_period*STEPS_PER_EPOCH))
    
  batchsize = 128;
  epoch = 1000;
  checkpoint_path = f"/home/saichaitanya/Chaitanya/Trained models /1percent data/{2**i}/cp.ckpt"
  # "/content/gdrive/My Drive/May3rd data model/ with hidden units",2**i,"/cp.ckpt"
  checkpoint_dir = os.path.dirname(checkpoint_path)

  # Create a callback that saves the model's weights
  cp_callback = tf.keras.callbacks.ModelCheckpoint(filepath=checkpoint_path,
                                                  save_weights_only=True,
                                                  verbose=1)

  # This may generate warnings related to saving the state of the optimizer.
  # These warnings (and similar warnings throughout this notebook)
  # are in place to discourage outdated usage, and can be ignored.

  model.compile(optimizer = opt, loss = loss, metrics = 'accuracy')
  # model.load_weights(checkpoint_path)
  model.fit(X_train,y_train,epochs = 1000,batch_size = 128,shuffle = True,use_multiprocessing = True,callbacks=[lr_scheduler,cp_callback])
  model.evaluate(X_test,y_test)

The Number of Hidden units used is:  16

Epoch 1: LearningRateScheduler setting learning rate to 0.0010000000474974513.
Epoch 1/1000
1356/1381 [============================>.] - ETA: 0s - loss: 0.0248 - accuracy: 0.8216
Epoch 1: saving model to /home/saichaitanya/Chaitanya/Trained models /1percent data/16/cp.ckpt
1381/1381 [==============================] - 3s 2ms/step - loss: 0.0247 - accuracy: 0.8220 - lr: 0.0010

Epoch 2: LearningRateScheduler setting learning rate to 0.0010000000474974513.
Epoch 2/1000
1363/1381 [============================>.] - ETA: 0s - loss: 0.0187 - accuracy: 0.8571
Epoch 2: saving model to /home/saichaitanya/Chaitanya/Trained models /1percent data/16/cp.ckpt
1381/1381 [==============================] - 2s 2ms/step - loss: 0.0187 - accuracy: 0.8574 - lr: 0.0010

Epoch 3: LearningRateScheduler setting learning rate to 0.0010000000474974513.
Epoch 3/1000
1372/1381 [============================>.] - ETA: 0s - loss: 0.0127 - accuracy: 0.8909
Epoch 3: saving model 

1381/1381 [==============================] - ETA: 0s - loss: 0.0055 - accuracy: 0.9253
Epoch 22: saving model to /home/saichaitanya/Chaitanya/Trained models /1percent data/16/cp.ckpt
1381/1381 [==============================] - 2s 2ms/step - loss: 0.0055 - accuracy: 0.9253 - lr: 0.0010

Epoch 23: LearningRateScheduler setting learning rate to 0.0010000000474974513.
Epoch 23/1000
1372/1381 [============================>.] - ETA: 0s - loss: 0.0054 - accuracy: 0.9258
Epoch 23: saving model to /home/saichaitanya/Chaitanya/Trained models /1percent data/16/cp.ckpt
1381/1381 [==============================] - 3s 2ms/step - loss: 0.0054 - accuracy: 0.9257 - lr: 0.0010

Epoch 24: LearningRateScheduler setting learning rate to 0.0010000000474974513.
Epoch 24/1000
1377/1381 [============================>.] - ETA: 0s - loss: 0.0054 - accuracy: 0.9258
Epoch 24: saving model to /home/saichaitanya/Chaitanya/Trained models /1percent data/16/cp.ckpt
1381/1381 [==============================] - 3s 2ms/s

1381/1381 [==============================] - 2s 2ms/step - loss: 0.0050 - accuracy: 0.9321 - lr: 0.0010

Epoch 44: LearningRateScheduler setting learning rate to 0.0010000000474974513.
Epoch 44/1000
1359/1381 [============================>.] - ETA: 0s - loss: 0.0051 - accuracy: 0.9304
Epoch 44: saving model to /home/saichaitanya/Chaitanya/Trained models /1percent data/16/cp.ckpt
1381/1381 [==============================] - 2s 2ms/step - loss: 0.0051 - accuracy: 0.9306 - lr: 0.0010

Epoch 45: LearningRateScheduler setting learning rate to 0.0010000000474974513.
Epoch 45/1000
1371/1381 [============================>.] - ETA: 0s - loss: 0.0050 - accuracy: 0.9329
Epoch 45: saving model to /home/saichaitanya/Chaitanya/Trained models /1percent data/16/cp.ckpt
1381/1381 [==============================] - 2s 2ms/step - loss: 0.0050 - accuracy: 0.9329 - lr: 0.0010

Epoch 46: LearningRateScheduler setting learning rate to 0.0010000000474974513.
Epoch 46/1000
1357/1381 [==========================

Epoch 65/1000
1359/1381 [============================>.] - ETA: 0s - loss: 0.0045 - accuracy: 0.9399
Epoch 65: saving model to /home/saichaitanya/Chaitanya/Trained models /1percent data/16/cp.ckpt
1381/1381 [==============================] - 2s 2ms/step - loss: 0.0045 - accuracy: 0.9398 - lr: 6.3763e-04

Epoch 66: LearningRateScheduler setting learning rate to 0.000637628196272999.
Epoch 66/1000
1369/1381 [============================>.] - ETA: 0s - loss: 0.0042 - accuracy: 0.9416
Epoch 66: saving model to /home/saichaitanya/Chaitanya/Trained models /1percent data/16/cp.ckpt
1381/1381 [==============================] - 2s 2ms/step - loss: 0.0042 - accuracy: 0.9417 - lr: 6.3763e-04

Epoch 67: LearningRateScheduler setting learning rate to 0.000637628196272999.
Epoch 67/1000
1377/1381 [============================>.] - ETA: 0s - loss: 0.0043 - accuracy: 0.9402
Epoch 67: saving model to /home/saichaitanya/Chaitanya/Trained models /1percent data/16/cp.ckpt
1381/1381 [======================

1381/1381 [==============================] - 2s 2ms/step - loss: 0.0038 - accuracy: 0.9428 - lr: 6.3763e-04

Epoch 87: LearningRateScheduler setting learning rate to 0.000637628196272999.
Epoch 87/1000
1374/1381 [============================>.] - ETA: 0s - loss: 0.0038 - accuracy: 0.9431
Epoch 87: saving model to /home/saichaitanya/Chaitanya/Trained models /1percent data/16/cp.ckpt
1381/1381 [==============================] - 2s 2ms/step - loss: 0.0038 - accuracy: 0.9431 - lr: 6.3763e-04

Epoch 88: LearningRateScheduler setting learning rate to 0.000637628196272999.
Epoch 88/1000
1364/1381 [============================>.] - ETA: 0s - loss: 0.0037 - accuracy: 0.9443
Epoch 88: saving model to /home/saichaitanya/Chaitanya/Trained models /1percent data/16/cp.ckpt
1381/1381 [==============================] - 3s 2ms/step - loss: 0.0037 - accuracy: 0.9443 - lr: 6.3763e-04

Epoch 89: LearningRateScheduler setting learning rate to 0.000637628196272999.
Epoch 89/1000
1352/1381 [=================

...

Epoch 906/1000
1380/1381 [============================>.] - ETA: 0s - loss: 6.6393e-04 - accuracy: 0.9795
Epoch 906: saving model to /home/saichaitanya/Chaitanya/Trained models /1percent data/1024/cp.ckpt
1381/1381 [==============================] - 20s 14ms/step - loss: 6.6387e-04 - accuracy: 0.9795 - lr: 1.1109e-05

Epoch 907: LearningRateScheduler setting learning rate to 1.110899665945908e-05.
Epoch 907/1000
1380/1381 [============================>.] - ETA: 0s - loss: 6.5855e-04 - accuracy: 0.9794
Epoch 907: saving model to /home/saichaitanya/Chaitanya/Trained models /1percent data/1024/cp.ckpt
1381/1381 [==============================] - 20s 14ms/step - loss: 6.5866e-04 - accuracy: 0.9794 - lr: 1.1109e-05

Epoch 908: LearningRateScheduler setting learning rate to 1.110899665945908e-05.
Epoch 908/1000
1380/1381 [============================>.] - ETA: 0s - loss: 6.5742e-04 - accuracy: 0.9797
Epoch 908: saving model to /home/saichaitanya/Chaitanya/Trained models /1percent data/1024/c

1381/1381 [==============================] - 20s 14ms/step - loss: 6.6595e-04 - accuracy: 0.9792 - lr: 1.1109e-05

Epoch 927: LearningRateScheduler setting learning rate to 1.110899665945908e-05.
Epoch 927/1000
1381/1381 [==============================] - ETA: 0s - loss: 6.6146e-04 - accuracy: 0.9795
Epoch 927: saving model to /home/saichaitanya/Chaitanya/Trained models /1percent data/1024/cp.ckpt
1381/1381 [==============================] - 20s 14ms/step - loss: 6.6146e-04 - accuracy: 0.9795 - lr: 1.1109e-05

Epoch 928: LearningRateScheduler setting learning rate to 1.110899665945908e-05.
Epoch 928/1000
1380/1381 [============================>.] - ETA: 0s - loss: 6.6353e-04 - accuracy: 0.9795
Epoch 928: saving model to /home/saichaitanya/Chaitanya/Trained models /1percent data/1024/cp.ckpt
1381/1381 [==============================] - 20s 14ms/step - loss: 6.6371e-04 - accuracy: 0.9795 - lr: 1.1109e-05

Epoch 929: LearningRateScheduler setting learning rate to 1.110899665945908e-05.
Ep

Epoch 947/1000
1379/1381 [============================>.] - ETA: 0s - loss: 6.5660e-04 - accuracy: 0.9796
Epoch 947: saving model to /home/saichaitanya/Chaitanya/Trained models /1percent data/1024/cp.ckpt
1381/1381 [==============================] - 20s 14ms/step - loss: 6.5628e-04 - accuracy: 0.9796 - lr: 1.1109e-05

Epoch 948: LearningRateScheduler setting learning rate to 1.110899665945908e-05.
Epoch 948/1000
1380/1381 [============================>.] - ETA: 0s - loss: 6.5908e-04 - accuracy: 0.9796
Epoch 948: saving model to /home/saichaitanya/Chaitanya/Trained models /1percent data/1024/cp.ckpt
1381/1381 [==============================] - 20s 14ms/step - loss: 6.6017e-04 - accuracy: 0.9796 - lr: 1.1109e-05

Epoch 949: LearningRateScheduler setting learning rate to 1.110899665945908e-05.
Epoch 949/1000
1380/1381 [============================>.] - ETA: 0s - loss: 6.5912e-04 - accuracy: 0.9795
Epoch 949: saving model to /home/saichaitanya/Chaitanya/Trained models /1percent data/1024/c

1381/1381 [==============================] - 20s 14ms/step - loss: 6.6072e-04 - accuracy: 0.9794 - lr: 8.8707e-06

Epoch 968: LearningRateScheduler setting learning rate to 8.870713827491272e-06.
Epoch 968/1000
1379/1381 [============================>.] - ETA: 0s - loss: 6.5566e-04 - accuracy: 0.9797
Epoch 968: saving model to /home/saichaitanya/Chaitanya/Trained models /1percent data/1024/cp.ckpt
1381/1381 [==============================] - 20s 14ms/step - loss: 6.5556e-04 - accuracy: 0.9797 - lr: 8.8707e-06

Epoch 969: LearningRateScheduler setting learning rate to 8.870713827491272e-06.
Epoch 969/1000
1381/1381 [==============================] - ETA: 0s - loss: 6.5939e-04 - accuracy: 0.9796
Epoch 969: saving model to /home/saichaitanya/Chaitanya/Trained models /1percent data/1024/cp.ckpt
1381/1381 [==============================] - 20s 14ms/step - loss: 6.5939e-04 - accuracy: 0.9796 - lr: 8.8707e-06

Epoch 970: LearningRateScheduler setting learning rate to 8.870713827491272e-06.
Ep

Epoch 988/1000
1379/1381 [============================>.] - ETA: 0s - loss: 6.5512e-04 - accuracy: 0.9801
Epoch 988: saving model to /home/saichaitanya/Chaitanya/Trained models /1percent data/1024/cp.ckpt
1381/1381 [==============================] - 20s 14ms/step - loss: 6.5503e-04 - accuracy: 0.9801 - lr: 8.8707e-06

Epoch 989: LearningRateScheduler setting learning rate to 8.870713827491272e-06.
Epoch 989/1000
1379/1381 [============================>.] - ETA: 0s - loss: 6.5170e-04 - accuracy: 0.9799
Epoch 989: saving model to /home/saichaitanya/Chaitanya/Trained models /1percent data/1024/cp.ckpt
1381/1381 [==============================] - 20s 14ms/step - loss: 6.5136e-04 - accuracy: 0.9799 - lr: 8.8707e-06

Epoch 990: LearningRateScheduler setting learning rate to 8.870713827491272e-06.
Epoch 990/1000
1378/1381 [============================>.] - ETA: 0s - loss: 6.5288e-04 - accuracy: 0.9800
Epoch 990: saving model to /home/saichaitanya/Chaitanya/Trained models /1percent data/1024/c

import matplotlib.pyplot as plt

## Model validation and plotting

In [129]:
i = 10
model = create_model(2**i)
model.load_weights(f"/home/saichaitanya/Chaitanya/Trained models /1percent data/{2**i}/cp.ckpt")
model.predict(X_test)

array([[0.18716541, 0.48456085],
       [0.63720226, 0.49796972],
       [0.10808465, 0.7143755 ],
       ...,
       [0.6889613 , 0.49660888],
       [0.6605398 , 0.49820906],
       [0.99445015, 0.4742612 ]], dtype=float32)

In [119]:
y_test

array([[0.199722  , 0.48816747],
       [0.64228165, 0.49793313],
       [0.10720997, 0.7140712 ],
       ...,
       [0.68169065, 0.49721468],
       [0.66150995, 0.49729591],
       [0.98849391, 0.48205711]])

In [131]:
"""
X_train, X_test, y_train, y_test = train_test_split(X_norm, y_norm, test_size=0.2, random_state=42)
for i in range(4,9,1):
  model2 = create_model(2**i)
  
  # model2.compile(optimizer = opt, loss = loss, metrics = 'accuracy')
  model2.load_weights(f"/content/gdrive/My Drive/May3rd data model/{2**i}/cp.ckpt")
  # model2.evaluate(X_test,y_test)
  y_pred = model2.predict(X_test)
  T_pred = y_pred[:,0]
  b_pred = y_pred[:,1]
  T_t = y_test[:,0]
  b_t = y_test[:,1] 
  T_pred.sort()
  T_t.sort()
  b_pred.sort()
  b_t.sort()
  fig = plt.figure(figsize=(15,4)) # creating a blank figure
  # creating a 2,1 grid
  gs = fig.add_gridspec(nrows=1,ncols=2)
  # plt.plot(T_pred)
  # plt.plot(T_t)
  # plt.plot(b_pred)
  # plt.plot(b_t)
  ax1 = fig.add_subplot(gs[:,0])
  ax2 = fig.add_subplot(gs[:,1])

  # plotting 
  ax1.plot(T_t,label="actual")
  ax1.plot(T_pred,label='pred')
  # ax1.plot(legend('actual','pred'))
  ax2.plot(b_t)
  ax2.plot(b_pred)
  ax1.plot(title="1 percent trajectory")
  ax2.plot(title="1 percent trajectory")
  # labelling
  ax1.set_xlabel('Number of points',size = 15)
  ax1.set_ylabel('Thrust',size = 15)
  ax2.set_xlabel('Number of points',size = 15)
  ax2.set_ylabel('beta',size = 15)
  """

'\nX_train, X_test, y_train, y_test = train_test_split(X_norm, y_norm, test_size=0.2, random_state=42)\nfor i in range(4,9,1):\n  model2 = create_model(2**i)\n  \n  # model2.compile(optimizer = opt, loss = loss, metrics = \'accuracy\')\n  model2.load_weights(f"/content/gdrive/My Drive/May3rd data model/{2**i}/cp.ckpt")\n  # model2.evaluate(X_test,y_test)\n  y_pred = model2.predict(X_test)\n  T_pred = y_pred[:,0]\n  b_pred = y_pred[:,1]\n  T_t = y_test[:,0]\n  b_t = y_test[:,1] \n  T_pred.sort()\n  T_t.sort()\n  b_pred.sort()\n  b_t.sort()\n  fig = plt.figure(figsize=(15,4)) # creating a blank figure\n  # creating a 2,1 grid\n  gs = fig.add_gridspec(nrows=1,ncols=2)\n  # plt.plot(T_pred)\n  # plt.plot(T_t)\n  # plt.plot(b_pred)\n  # plt.plot(b_t)\n  ax1 = fig.add_subplot(gs[:,0])\n  ax2 = fig.add_subplot(gs[:,1])\n\n  # plotting \n  ax1.plot(T_t,label="actual")\n  ax1.plot(T_pred,label=\'pred\')\n  # ax1.plot(legend(\'actual\',\'pred\'))\n  ax2.plot(b_t)\n  ax2.plot(b_pred)\n  ax1.plot(